In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline

In [2]:
df2 = pd.read_csv('../orga-datos/datos_preprocesados/fiuba_6_avisos_detalle.csv')

In [3]:
cantidad_de_palabras_que_quiero = 500
porcetaje=0.01
similitud=0.3

In [4]:
#from descripcion_parser import parse

In [5]:
stopswords="a acá ahí ajena ajeno ajenos ajenas al algo algún algúna algúno algúnos algúnas allá allí ambos ante antes aquel aquella aquello aquellos aquellas aquí arriba así atrás aun aunque bajo bastante bien cabe cada casi cierto cierta ciertos ciertas como con conmigo conseguimos conseguir consigo consigue consiguen consigues contigo contra cual cuales cualquier cualquiera cualquieras cuancuando cuanto cuantas cuanta cuantos de dejar del demás demasiada demasiadas demasiado demasiados dentro desde donde dos e el él ella ellas ello ellos empleáis emplean emplear empleas empleo en encima entonces entre era eras eramos eran eres es esa ese eso esos esas esta estas estaba estado estáis estamos están estar este esto estos estas estoy etc fin fue fueron fui fuimos gueno ha hace haces hacéis hacemos hacen hacer hacia hago hasta incluso intenta intentas intentáis intentamos intentan intentar intento ir jamás junto juntos la lo las los largo más me menos mi mis mía mías mientras mío míos misma mismo mismos mismas modo mucha muchas muchísima muchísimos muchísimo muchísimas mucho muchos muy nada ni ningún ningúna ningúnas ningúno ningúnos no nos nosotras nosotros nuestra nuestro nuestros nuestras nunca o os otra otro otras otros para parecer pero poca poco pocas pocos podéis podemos poder podría podrías podríais podríamos podrían por por qué porque primero puede pueden puedo pues que qué querer quién quiénes quienesquiera quienquiera quizá quizás sabe sabes saben sabéis sabemos saber se según ser si sí siempre siendo sin sino so sobre sois solamente solo sólo somos soy sr sra sres sta su sus suyo suyos suya suyas tal tales también tampoco tan tanta tanto tantos tantas te tenéis tenemos tener tengo ti tiempo tiene tienen todo todos toda todas tomar trabaja trabajo trabajáis trabajamos trabajan trabajar trabajas tras tú tu tus tuya tuyas tuyo tuyos último ultimo un una unas uno unos usa usas usáis usamos usan usar uso usted ustedes va van vais valor vamos varias varios vaya verdadera vosotras vosotros voy vuestra vuestros vuestro vuestras y ya yo como donde requisitos requisito buscamos importante buenas muy grandes beneficios encontramos busqueda zona "
stopswords=stopswords.split()

In [6]:
from stop_words import ESP_STOP_WORDS as BASE_STOP_WORDS, ENG_STOP_WORDS

stopswords = set([ 
 'preferentemente'
]).union(BASE_STOP_WORDS.union(ENG_STOP_WORDS).union(stopswords))

In [7]:
from html.parser import HTMLParser

class MLStripper(HTMLParser):
    def __init__(self):
        self.reset()
        self.strict = False
        self.convert_charrefs= True
        self.fed =[]
    def handle_data(self, d):
        self.fed.append(d)
    def get_data(self):
        return ' '.join(self.fed)

def strip_html_tags(html):
    s = MLStripper()
    s.feed(html)
    return s.get_data()

def foldear_simbolos(cadena):
    ''' 
    Elimina los acentos de una cadena en minúsculas. 
    Ej: descripción -> descripcion
    '''
    acentos = {'á':'a', 'é':'e', 'í':'i', 'ó':'o', 'ú':'u', 'ü':'u'}
    cadena = list(cadena)
    for i in range(len(cadena)):
        if not cadena[i].isalpha():
            cadena[i] = ' '
        else:
            cadena[i] = acentos.get(cadena[i], cadena[i])
    
    return ''.join(cadena)

def es_palabra_inutil(palabra):
    '''
    Devuelve True si palabra es una palabra que no aporta
    contenido.
    '''
    return palabra in set(stopswords)

def parse(descripcion):
    '''
    Normaliza una descripción de aviso del set de datos de Navent.
    Elimina tags HTML, pasa a minúsculas y la convierte a tokens.
    '''
    resultado = []
    
    for palabra in strip_html_tags(descripcion).lower().split():
        if es_palabra_inutil(palabra):
            continue
            
        resultado.extend(foldear_simbolos(palabra).split())
    
    return ' '.join(resultado)

In [8]:
words={}
def total_freq(description):
    #it create a dictionary with the following format (word : #appearances in the sum of descriptions)
    words_processed=[]
    for word in description.split():
        if word not in words_processed:
            words[word] = words.get(word, 0) + 1
            words_processed.append(word)
    return description

factor = df2["descripcion"].count()*porcetaje #aca va el porcentaje que se quiere tener

def remove_less_frequent_words(description):
    # we will remove all the words which appear in less than 1% of the descrptions
    result = []
    for word in description.lower().split():
        if words[word] < factor:
            continue
            
        result.append(word)
    
    return ' '.join(result)

In [9]:
df2["descripcion"]=df2["descripcion"].map(parse)

In [10]:
import Stemmer

stemmer = Stemmer.Stemmer('spanish')
raices={}

def conseguir_raices(descripcion):
    
    for palabra in descripcion.lower().split():
        if palabra in raices:
            continue
        p=stemmer.stemWords([palabra])
        raices[palabra]=p[0]
    return descripcion
    
def reempazar_por_raiz(description):
    
    result = []
    for word in description.lower().split():
        word=raices[word]    
        result.append(word)
    
    return ' '.join(result)
    

In [11]:
df2["descripcion"]=df2["descripcion"].map(conseguir_raices)

In [12]:
df2["descripcion"]=df2["descripcion"].map(reempazar_por_raiz)

In [13]:
df2["descripcion"]=df2["descripcion"].map(total_freq)

In [14]:
df2["descripcion"] = df2["descripcion"].map(remove_less_frequent_words)

In [15]:
# sudo pip3 install datasketch -U

from datasketch import MinHash

#te da lasdescripciones que tengan semejanza mayor a 0.3 (este es un buen factor)

data1=df2["descripcion"][43]
data1=data1.split()
    
def jac(texto):
    
    data2=texto.split()
    m1, m2 = MinHash(), MinHash()
    for d in data1:
        m1.update(d.encode('utf8'))
    for d in data2:
        m2.update(d.encode('utf8'))
    if m1.jaccard(m2)> similitud:     
        print("Estimated Jaccard for data1 and data2 is", m1.jaccard(m2))
        print(" ".join(data2))

d=df2["descripcion"].map(jac)  
print("ok")

Estimated Jaccard for data1 and data2 is 1.0
busqued anal credit jr sector entid bancari posicion respons ser analisis evaluacion propuest credit participacion conjunt credit administracion carter requisit postulat si sos profesional estudi avanz carrer cienci econom licenci administracion ingenier ten actitud proactiv manej relacion interpersonal equip orientacion result cont manej paquet offic valor conoc segment experient laboral analisis balanc riesg credit benefici condicion contratacion lun viern jorn tim trabaj microcentr empres ofrec condicion contratacion
Estimated Jaccard for data1 and data2 is 0.3046875
busqued un estudi carrer cienci econom desempeñ anal junior administracion finanz principal tar ser participacion proces pag proveedor emision valor fisic y o electron confeccion legaj pag proveedor cambi gast estructur confeccion administracion fond fij pag viaj otras busqued orient person agil dinam capac organizacion relacion interpersonal aprend desarroll are ser excluyen

In [16]:
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
from scipy.sparse.csr import csr_matrix #need this if you want to save tfidf_matrix

#tf = TfidfVectorizer(input='/home/iglesias/test', analyzer='word', ngram_range=(1,6),
#min_df = 0, stop_words = 'english', sublinear_tf=True)
tf = TfidfVectorizer() 
tfidf_matrix =  tf.fit_transform(df2["descripcion"])


feature_names = tf.get_feature_names()

"""doc =0
feature_index = tfidf_matrix[doc,:].nonzero()[1]
tfidf_scores = zip(feature_index, [tfidf_matrix[doc, x] for x in feature_index])
for w, s in [(feature_names[i], s) for (i, s) in tfidf_scores]:
  print(w, s)
"""
factor=df2["descripcion"].count()

scores=[]
dos_mil_palabras_importantes = []
for doc in range(factor):
    feature_index = tfidf_matrix[doc,:].nonzero()[1]
    tfidf_scores = zip(feature_index, [tfidf_matrix[doc, x] for x in feature_index])
    for w, s in [(feature_names[i], s) for (i, s) in tfidf_scores]:
      if not w in dos_mil_palabras_importantes:   
          scores.append(-s)
          dos_mil_palabras_importantes.append(w)  

import heapq

scores_mas_importantes=[]
heapq.heapify(scores)

#fijarse de tener esta cantidad de palabras sino rompe

for x in range(cantidad_de_palabras_que_quiero):
    scores_mas_importantes.append(heapq.heappop(scores))

dos_mil_palabras_importantes = []    
    
for doc in range(df2["descripcion"].count()):
    feature_index = tfidf_matrix[doc,:].nonzero()[1]
    tfidf_scores = zip(feature_index, [tfidf_matrix[doc, x] for x in feature_index])
    for w, s in [(feature_names[i], s) for (i, s) in tfidf_scores]:
      if -s in scores_mas_importantes:   
          dos_mil_palabras_importantes.append(w) 
            
print("ok")  


ok


In [17]:
data2=df2["descripcion"][43]
data2#nuevo

'busqued anal credit jr sector entid bancari posicion respons ser analisis evaluacion propuest credit participacion conjunt credit administracion carter requisit postulat si sos profesional estudi avanz carrer cienci econom licenci administracion ingenier ten actitud proactiv manej relacion interpersonal equip orientacion result cont manej paquet offic valor conoc segment experient laboral analisis balanc riesg credit benefici condicion contratacion lun viern jorn tim trabaj microcentr empres ofrec condicion contratacion'

In [18]:
def mas_impor(description):
    
    resultado = []
    
    for palabra in description.lower().split():
        if palabra not in dos_mil_palabras_importantes:
            continue
            
        resultado.append(palabra)
    
    return ' '.join(resultado)

In [19]:
df2["descripcion"]=df2["descripcion"].map(mas_impor)

In [20]:
data2=df2["descripcion"][43]
data2#nuevo

'credit sector entid bancari posicion evaluacion propuest credit conjunt credit postulat si sos profesional licenci actitud proactiv interpersonal cont valor segment balanc credit benefici jorn tim trabaj microcentr'

In [21]:
from datasketch import MinHash

data1=df2["descripcion"][43]
data1=data1.split()
    
def jac(texto):
    
    data2=texto.split()
    m1, m2 = MinHash(), MinHash()
    for d in data1:
        m1.update(d.encode('utf8'))
    for d in data2:
        m2.update(d.encode('utf8'))
    if m1.jaccard(m2)> similitud:     
        print("Estimated Jaccard for data1 and data2 is", m1.jaccard(m2))
        print(" ".join(data2))

d=df2["descripcion"].map(jac)
print("ok")

Estimated Jaccard for data1 and data2 is 1.0
credit sector entid bancari posicion evaluacion propuest credit conjunt credit postulat si sos profesional licenci actitud proactiv interpersonal cont valor segment balanc credit benefici jorn tim trabaj microcentr
Estimated Jaccard for data1 and data2 is 0.5859375
cobranz compañi financ grup reconoc entid bancari posicion grup postulat si sos profesional actitud proactiv interpersonal cont valor jorn tim trabaj microcentr
Estimated Jaccard for data1 and data2 is 0.3203125
industrial posicion deseabl cont comprob posicion encontr vigent futur desarroll desarroll plan propuest valor proactiv interpersonal jorn trabaj tim trabaj pod envi cv via
Estimated Jaccard for data1 and data2 is 0.3125
industrial posicion cont comprob posicion encontr vigent futur desarroll distribucion desarroll plan propuest valor proactiv interpersonal jorn trabaj tim trabaj
ok


In [22]:
data1=df2["descripcion"][5]
print(data1)

cajer requier secundari complet habil atencion trat client valor perfil proactiv resid plat


In [23]:
from datasketch import MinHash

data1=df2["descripcion"][5]
data1=data1.split()
    
def jac(texto):
    
    data2=texto.split()
    m1, m2 = MinHash(), MinHash()
    for d in data1:
        m1.update(d.encode('utf8'))
    for d in data2:
        m2.update(d.encode('utf8'))
    if m1.jaccard(m2)> similitud:     
        print("Estimated Jaccard for data1 and data2 is", m1.jaccard(m2))
        print(" ".join(data2))

d=df2["descripcion"].map(jac)  
print("ok")

Estimated Jaccard for data1 and data2 is 1.0
cajer requier secundari complet habil atencion trat client valor perfil proactiv resid plat
Estimated Jaccard for data1 and data2 is 0.3046875
caden supermerc cajer repositor cont secundari complet informat valor caj atencion client mercaderi interes cont tim eventual
Estimated Jaccard for data1 and data2 is 0.328125
caden supermerc cajer cont secundari complet sex masculin edad caj sanitari tim valor caj atencion client interes cont tim
Estimated Jaccard for data1 and data2 is 0.3125
tim secundari complet basic electron acces basic contact atencion client habil trat interpersonal negociacion proactiv compromis client
Estimated Jaccard for data1 and data2 is 0.34375
multinacional rubr coordinacion comunic client instalacion carg dat perfil atencion client secundari complet tim client habil empati resolut enfoc proactiv
Estimated Jaccard for data1 and data2 is 0.4609375
desarroll client maquinari comercial desarroll client requier perfil pers

In [24]:
data1=df2["descripcion"][10]
print(data1)

impuest belgran capital federal junior impuest client global capital federal belgran encuentr contador impuest contabl basic imposit form are encuentr impuest nacional informacion impuest basic ingres imposit contabl cont tim trabaj interes profesional global postulat contador resident capital federal edad salari complet salari tim trabaj belgran are impuest interes envi cv com ar referent junior impuest


In [25]:
from datasketch import MinHash

data1=df2["descripcion"][10]
data1=data1.split()
    
def jac(texto):
    
    data2=texto.split()
    m1, m2 = MinHash(), MinHash()
    for d in data1:
        m1.update(d.encode('utf8'))
    for d in data2:
        m2.update(d.encode('utf8'))
    if m1.jaccard(m2)> similitud:     
        print("Estimated Jaccard for data1 and data2 is", m1.jaccard(m2))
        print(" ".join(data2))

d=df2["descripcion"].map(jac)
print("ok")

Estimated Jaccard for data1 and data2 is 1.0
impuest belgran capital federal junior impuest client global capital federal belgran encuentr contador impuest contabl basic imposit form are encuentr impuest nacional informacion impuest basic ingres imposit contabl cont tim trabaj interes profesional global postulat contador resident capital federal edad salari complet salari tim trabaj belgran are impuest interes envi cv com ar referent junior impuest
Estimated Jaccard for data1 and data2 is 0.328125
crecimient seleccion contabl contador nacional desempeñ contabl bancari contabl fond caj requier are contabl contabl capital federal trabaj tim interes envi cv
Estimated Jaccard for data1 and data2 is 0.328125
form trabaj trat client intern cont atencion valor are legal cobranz rubr inmobiliari sex edad tim resident capital federal interes envi cv
Estimated Jaccard for data1 and data2 is 0.3359375
nacional crecimient seleccion contabl contador nacional desempeñ bancari contabl requier contabl